In [109]:
import pandas as pd 
import numpy as np 
from sklearn.datasets import fetch_california_housing 
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.model_selection import KFold,cross_val_score,GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

In [26]:
data = fetch_california_housing()

In [27]:
data

{'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
           37.88      , -122.23      ],
        [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
           37.86      , -122.22      ],
        [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
           37.85      , -122.24      ],
        ...,
        [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
           39.43      , -121.22      ],
        [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
           39.43      , -121.32      ],
        [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
           39.37      , -121.24      ]]),
 'target': array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894]),
 'frame': None,
 'target_names': ['MedHouseVal'],
 'feature_names': ['MedInc',
  'HouseAge',
  'AveRooms',
  'AveBedrms',
  'Population',
  'AveOccup',
  'Latitude',
  'Longitude'],
 'DESCR': '.. _california_housing_dataset:\n

In [28]:
df_input = pd.DataFrame(data.data,columns=data.feature_names)

In [29]:
df_target = pd.DataFrame(data.target,columns=data.target_names)

In [30]:
df = pd.concat([df_input,df_target],axis=1)

In [31]:
df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [36]:
X = df.iloc[:,:-1]
y = df["MedHouseVal"]

In [87]:
kf = KFold(n_splits=10,random_state=32,shuffle=True)

In [88]:
#### calculating cross_val_score for linear regression 
scores=cross_val_score(LinearRegression(),X,y,cv=kf,scoring="r2")

In [89]:
scores

array([0.60691782, 0.56776398, 0.60058565, 0.60776976, 0.58616007,
       0.61622877, 0.54848483, 0.63472942, 0.62870028, 0.60118153])

In [92]:
# calculating the mean of the scores, 
scores_mean = np.mean(scores)
scores_std = np.std(scores)

In [95]:
scores_std,scores_mean

(0.025074836634283497, 0.5998522115018672)

LETS DO IT IN A BETTER WAY 

In [105]:
def modelCalc(X,y,model):
    mse,r2score = [],[]
    for train_idx,test_idx in kf.split(X):
        X_train,X_test = X.iloc[train_idx],X.iloc[test_idx]
        y_train,y_test = y.iloc[train_idx],y.iloc[test_idx]

        model.fit(X_train,y_train)
        pred = model.predict(X_test)
        mse.append(mean_squared_error(pred,y_test))
        r2score.append(r2_score(pred,y_test))

    return mse,r2score

        

In [106]:
mse_lr,r2score_lr = modelCalc(X,y,LinearRegression())

In [108]:
param_grid = {
    'max_depth': [3, 5, 10, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

In [110]:
grid_search = GridSearchCV(
    estimator=DecisionTreeRegressor(random_state=42),
    param_grid=param_grid,
    scoring='neg_mean_squared_error',
    cv=kf,
    n_jobs=-1
)

In [112]:
mse_dtc,r2_dtc = modelCalc(X,y,grid_search)

In [113]:
mse_dtc,r2_dtc

([0.3901106902906788,
  0.39324733888310986,
  0.3666069485996953,
  0.35963707712759,
  0.4179598942901163,
  0.3900288273117894,
  0.42342125084940946,
  0.3413925759257137,
  0.3571502714338084,
  0.3924305368784107],
 [0.6483893754514976,
  0.6411662855526088,
  0.6464436470165885,
  0.6399942692576459,
  0.6195891671422593,
  0.6489848935687053,
  0.6235741273638271,
  0.7060037301808519,
  0.657053726705358,
  0.6379923667133616])